<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Package installations

In [30]:
# pip installs
!pip install firebase
!pip install gradio
!pip install paho-mqtt

#================================= make sure all pip installs are above this line ============================================

# import to clear the installation code output
from IPython.display import clear_output
clear_output()

In [29]:
#imports
import gradio as gr
import json
import time
from firebase import firebase
import paho.mqtt.client as mqtt

# Admin panel functions

# Index and search engine

# Sensor data caching and processing

In [ ]:
# Data proccessing
#def process_data(data):

db_url = "https://project-pheonix-39eef-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(db_url,None)

# Data saving in DB
def send_to_db(path, data):
  FBconn.put('/',f'/fake/{path}',data)
  return

# create an mqtt connection
def on_connect(client, userdata, flags, rc):
  if rc == 0:
    print("Connected to MQTT Broker!\nSubscribing to topics")

    # Subscribe to the relevant topics
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")

    print("Successfully subscribed to topics!")
  else:
    print(f"Failed to connect, return code {rc}")

def on_disconnect(client, userdata, rc):
  if rc != 0:
    for i in range(5):
      print(f"Unexpected disconnection (error code: {rc}). Attempting to reconnect number {i + 1} in 5 seconds...")
      time.sleep(5)
      try:
        client.reconnect()
      except Exception as e:
        print(f"Reconnection attempt failed: {e}")

def on_message(client, userdata, msg):

  topic = msg.topic
  payload = msg.payload.decode('utf-8')  # Decode the byte string to a string

  print(f"Received JSON message on topic '{topic}': {payload}")
  entry = None
  try:
    sensor_data = json.loads(payload)
    print(f"Parsed JSON data: {sensor_data}")

    if topic == "braude/D106/indoor":
      send_to_db(f"indoor/{int(time.time())}", sensor_data)

    elif topic == "braude/D106/outdoor":
      send_to_db(f"outdoor/{int(time.time())}", sensor_data)

  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Problematic payload: {payload}")

# connect to the MQTT publisher
client = mqtt.Client()
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message
client.connect("broker.hivemq.com", 1883, keepalive = 600)
client.loop_start()

time.sleep(5)
client.loop_stop()
client.disconnect()

<ipython-input-33-e914c0f3eacf>:57: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected to MQTT Broker!
Subscribing to topics
Successfully subscribed to topics!
Received JSON message on topic 'braude/D106/indoor': {"Temperature": 20.1, "Pressure": 992.8, "Humidity": 20.1, "Distance": 38.1}
Parsed JSON data: {'Temperature': 20.1, 'Pressure': 992.8, 'Humidity': 20.1, 'Distance': 38.1}
Received JSON message on topic 'braude/D106/indoor': {"Temperature": 31.7, "Pressure": 1009.3, "Humidity": 31.7, "Distance": 13.1}
Parsed JSON data: {'Temperature': 31.7, 'Pressure': 1009.3, 'Humidity': 31.7, 'Distance': 13.1}
Received JSON message on topic 'braude/D106/outdoor': {"DLIGHT": 407.2}
Parsed JSON data: {'DLIGHT': 407.2}
Received JSON message on topic 'braude/D106/indoor': {"Temperature": 15.7, "Pressure": 1002.8, "Humidity": 15.7, "Distance": 19.5}
Parsed JSON data: {'Temperature': 15.7, 'Pressure': 1002.8, 'Humidity': 15.7, 'Distance': 19.5}
Received JSON message on topic 'braude/D106/outdoor': {"DLIGHT": 310.3}
Parsed JSON data: {'DLIGHT': 310.3}
Received JSON message 

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

# Screens

In [37]:
import plotly.graph_objects as go
from datetime import datetime
# Sensor data pulling from DB
data = FBconn.get('/fake/indoor',None)
data_keys = list(data.keys())
data_values = list(data.values())
readable_times = [datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S') for ts in data_keys]

#values for each indoor sensor
data_values_Distance = [value['Distance'] for value in data_values]
data_values_Temperature = [value['Temperature'] for value in data_values]
data_values_Humidity = [value['Humidity'] for value in data_values]
data_values_Pressure = [value['Pressure'] for value in data_values]

# Data visualization
def plot_graph(name):
    val_arr = [value[name] for value in data_values]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=readable_times, y=val_arr, mode='lines+markers', name=name))
    fig.update_layout(title='Sensor {} Over Time'.format(name), xaxis_title='Time', yaxis_title=name)
    return fig

# Gradio interface
gr.Interface(fn=plot_graph, inputs=gr.Dropdown(['Distance','Temperature','Humidity','Pressure']), outputs=gr.Plot(label="Graph")).launch(inline=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63cd48d5b3aec45fcb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
